In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn.cluster import KMeans
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
import math
import random
import time
from tkinter import *
da=pd.read_csv('C:/Users/User/Documents/MachineLearning/MachineLearning/assi/ML/ML/iris.csv')
da.head()
labels = da['Species'].values.tolist()
labels
da.dropna()
data=da.drop(['Species'], axis=1)
data
list=data.values.tolist()
list
def euclidean(centroid, data):
    sum1=[]
    if len(centroid) == len(data)+1:
        centroid=centroid[1:]
    for i in range (0, len(data)):
        sum1.append((centroid[i]- data[i])**2)
    
    
    euclidean= math.sqrt(sum(sum1))
    return euclidean
def jaccard(centroid, data):
    jacSummin=[]
    jacSummax = []
    if len(centroid) == 5:
        centroid = centroid[1:]
    for i in range(len(data)):
        jacSummin.append(min(centroid[i],data[i]))
        jacSummax.append(max(centroid[i],data[i]))
    return 1-(sum(jacSummin)/sum(jacSummax))
def cosine(centroid, data):
    if len(centroid) == 5:
        centroid = centroid[1:]
    
    #print(centroid)
    x1 = centroid
    x1 = np.array(x1)
    x1 = x1.reshape(1,-1)
    y1 = np.array(data)
    y1 = y1.reshape(1,-1)
    #print(x1,y1)
    ans = cosine_similarity(x1,y1)
    return 1-ans[0][0]
def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0] # remove the header
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    return dataset
def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])
def distance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    sumOfSquares = 0
    for i in range(0, len(instance1)-1):
        sumOfSquares += (instance1[i+1] - instance2[i])**2
    return sumOfSquares

def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes)
    for instance in instanceList:
        for i in range(0, numAttributes):
            means[i+1] += instance[i]
   
    for i in range(0, numAttributes):
        means[i+1] /= float(numInstances)
    return tuple(means)

def assign(instance, centroids, dist):
    minDistance = dist(centroids[0],instance)
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = dist(centroids[i], instance )
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex
def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

def assignAll(instances, centroids, labels, dist):
    clusters = createEmptyListOfLists(len(centroids))
    classclusters = createEmptyListOfLists(len(centroids))
    i=0
    for instance in instances:
        clusterIndex = assign(instance, centroids, dist)
        clusters[clusterIndex].append(instance)
        classclusters[clusterIndex].append(labels[i])
        i=i+1
    return clusters, classclusters

def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "centroid" + str(i)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

def kmeans(instances, k, dist,labels, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances

    iteration = 0
    while (centroids != prevCentroids):
        iteration += 1
        clusters, required_labels = assignAll(instances, centroids, labels, dist)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, dist)

    print('Number of iterations are ', iteration)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = required_labels
    return result

def computeWithinss(clusters, centroids, dist):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += dist(centroid, instance)
    return result

# Repeats k-means clustering method n times, and returns the clustering
# with the smallest withinss
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i )
        trialClustering = kmeans(instances, k)
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering
def printTable(instances):
    for instance in instances:
        if instance != None:
            line = instance[0] + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print(line)

def extractAttribute(instances, index):
    result = []
    for instance in instances:
        result.append(instance[index])
    return result

def paintCircle(canvas, xc, yc, r, color):
    canvas.create_oval(xc-r, yc-r, xc+r, yc+r, outline=color)

def paintSquare(canvas, xc, yc, r, color):
    canvas.create_rectangle(xc-r, yc-r, xc+r, yc+r, fill=color)

def drawPoints(canvas, instances, color, shape):
    random.seed(0)
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    scaleX = float(width - 2*margin) / (maxX - minX)
    scaleY = float(height - 2*margin) / (maxY - minY)
    for instance in instances:
        x = 5*(random.random()-0.5)+margin+(instance[1]-minX)*scaleX
        y = 5*(random.random()-0.5)+height-margin-(instance[2]-minY)*scaleY
        if (shape == "square"):
            paintSquare(canvas, x, y, 5, color)
        else:
            paintCircle(canvas, x, y, 5, color)
    canvas.update()

def connectPoints(canvas, instances1, instances2, color):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    scaleX = float(width - 2*margin) / (maxX - minX)
    scaleY = float(height - 2*margin) / (maxY - minY)
    for p1 in instances1:
        for p2 in instances2:
            x1 = margin + (p1[1]-minX)*scaleX
            y1 = height - margin - (p1[2]-minY)*scaleY
            x2 = margin + (p2[1]-minX)*scaleX
            y2 = height - margin - (p2[2]-minY)*scaleY
            canvas.create_line(x1, y1, x2, y2, fill=color)
    canvas.update()

def mergeClusters(clusters):
    result = []
    for cluster in clusters:
        result.extend(cluster)
    return result

def prepareWindow(instances):
    width = 500
    height = 500
    margin = 50
    root = Tk()
    canvas = Canvas(root, width=width, height=height, background="white")
    canvas.pack()
    canvas.data = {}
    canvas.data["margin"] = margin
    setBounds2D(canvas, instances)
    paintAxes(canvas)
    canvas.update()
    return canvas

def setBounds2D(canvas, instances):
    attributeX = extractAttribute(instances, 1)
    attributeY = extractAttribute(instances, 2)
    canvas.data["minX"] = min(attributeX)
    canvas.data["minY"] = min(attributeY)
    canvas.data["maxX"] = max(attributeX)
    canvas.data["maxY"] = max(attributeY)

def paintAxes(canvas):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    canvas.create_line(margin/2, height-margin/2, width-5, height-margin/2,
                       width=2, arrow=LAST)
    canvas.create_text(margin, height-margin/4,
                       text=str(minX), font="Sans 11")
    canvas.create_text(width-margin, height-margin/4,
                       text=str(maxX), font="Sans 11")
    canvas.create_line(margin/2, height-margin/2, margin/2, 5,
                       width=2, arrow=LAST)
    canvas.create_text(margin/4, height-margin,
                       text=str(minY), font="Sans 11", anchor=W)
    canvas.create_text(margin/4, margin,
                       text=str(maxY), font="Sans 11", anchor=W)
    canvas.update()


def showDataset2D(instances):
    canvas = prepareWindow(instances)
    paintDataset2D(canvas, instances)

def paintDataset2D(canvas, instances):
    canvas.delete(ALL)
    paintAxes(canvas)
    drawPoints(canvas, instances, "blue", "circle")
    canvas.update()

def showClusters2D(clusteringDictionary):
    clusters = clusteringDictionary["clusters"]
    centroids = clusteringDictionary["centroids"]
    withinss = clusteringDictionary["withinss"]
    canvas = prepareWindow(mergeClusters(clusters))
    paintClusters2D(canvas, clusters, centroids,
                    "Withinss: %.1f" % withinss)

def paintClusters2D(canvas, clusters, centroids, title=""):
    canvas.delete(ALL)
    paintAxes(canvas)
    colors = ["blue", "red", "green", "brown", "purple", "orange"]
    for clusterIndex in range(len(clusters)):
        color = colors[clusterIndex%len(colors)]
        instances = clusters[clusterIndex]
        centroid = centroids[clusterIndex]
        drawPoints(canvas, instances, color, "circle")
        if (centroid != None):
            drawPoints(canvas, [centroid], color, "square")
        connectPoints(canvas, [centroid], instances, color)
    width = canvas.winfo_reqwidth()
    canvas.create_text(width/2, 20, text=title, font="Sans 14")
    canvas.update()
clustering = kmeans(list, 3, cosine,labels, True)
printTable(clustering["centroids"])
import statistics 
from statistics import mode 
def sse(clustering, centroids):
    i=0
    SSE=0
    for clust in clustering:
        for data in clust:
            SSE=SSE+distance(centroids[i], data)
        i=i+1
    return SSE
def accuracy(clustering):
    correct_pred = 0
    for clust in clustering['labels']:
        for label in clust:
            if label == mode(clust):
                correct_pred+=1
    return correct_pred/150
#print(cluster)
print('Cosine_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Cosine_Accuracy is ",accuracy(clustering))
clustering = kmeans(list, 3, jaccard,labels, True)
printTable(clustering["centroids"])
#print(cluster)
print('jaccard_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("jaccard_Accuracy is ",accuracy(clustering))
clustering = kmeans(list, 3, euclidean,labels, True)
printTable(clustering["centroids"])
#print(cluster)
print('eucledian_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("eucledian_Accuracy is ",accuracy(clustering))
def kmeans_sse(instances, k, dist,labels, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = [] 
    new_sse= 99999 
    prev_sse=1000000 
    if animation:
        delay = 1.0 

        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
        
    iteration = 0
    while (new_sse < prev_sse):
        iteration += 1
        clusters, required_labels = assignAll(instances, centroids, labels, dist)
       
        prev_sse = new_sse
        centroids = computeCentroids(clusters)
        new_sse= sse(clusters, centroids)
        withinss = computeWithinss(clusters, centroids, dist)
        
    print('Number of iterations are ', iteration)
    print('prev sse', prev_sse)
    print('new sse', new_sse)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = required_labels
    return result
before= datetime.now()
clustering = kmeans_sse(list, 3, cosine,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("time taken is ", after- before)
#print(cluster)
print('Cosine_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Cosine_Accuracy is ",accuracy(clustering))

before= datetime.now()
clustering = kmeans_sse(list, 3, jaccard,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("time taken is ", after- before)
#print(cluster)
print('jaccard_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("jaccard_Accuracy is ",accuracy(clustering))

before= datetime.now()
clustering = kmeans_sse(list, 3, euclidean,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("time taken is ", after- before)
#print(cluster)
print('eucledian_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("eucledian_Accuracy is ",accuracy(clustering))
def kmeans_100(instances, k, dist,labels, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances

    iteration = 0
    while (iteration<100):
        iteration +=1
        clusters, required_labels = assignAll(instances, centroids, labels, dist)
        
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, dist)

    print('Number of iterations are ', iteration)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = required_labels
    return result
before= datetime.now()
clustering = kmeans_100(list, 3, cosine,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("time taken is ", after- before)
#print(cluster)
print('cosine_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("cosine_Accuracy is ",accuracy(clustering))
before= datetime.now()
clustering = kmeans_100(list, 3, jaccard,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("time taken is ", after- before)
#print(cluster)
print('jaccard_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("jaccard_Accuracy is ",accuracy(clustering))
before= datetime.now()
clustering = kmeans_100(list, 3, euclidean,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("time taken is ", after- before)
#print(cluster)
print('eucledian_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("eucledian_Accuracy is ",accuracy(clustering))


Number of iterations are  11
centroid0	6.54 2.96 5.50 1.99 
centroid1	5.94 2.76 4.21 1.30 
centroid2	5.01 3.43 1.46 0.25 
Cosine_SSE is  91.98930917874398
Cosine_Accuracy is  0.9733333333333334
Number of iterations are  11
centroid0	6.61 3.00 5.39 1.92 
centroid1	5.01 3.43 1.46 0.25 
centroid2	5.69 2.67 4.12 1.27 
jaccard_SSE is  84.24288455008487
jaccard_Accuracy is  0.8933333333333333
Number of iterations are  9
centroid0	5.01 3.43 1.46 0.25 
centroid1	6.85 3.08 5.72 2.05 
centroid2	5.88 2.74 4.39 1.43 
eucledian_SSE is  78.85566582597731
eucledian_Accuracy is  0.8866666666666667
Number of iterations are  7
prev sse 91.9893091787439
new sse 91.9893091787439
centroid0	6.54 2.96 5.50 1.99 
centroid1	5.01 3.43 1.46 0.25 
centroid2	5.94 2.76 4.21 1.30 
time taken is  0:00:07.839448
Cosine_SSE is  91.9893091787439
Cosine_Accuracy is  0.9733333333333334
Number of iterations are  5
prev sse 79.09734974533113
new sse 79.09734974533113
centroid0	6.87 3.09 5.72 2.07 
centroid1	5.89 2.74 4.41 1